# Lab 04: Image Matching and Image Stitching

In this lab, you'll implement and play with the algorithms taught in course 5 and 6. 

- Student Name: 你的名字
- Student ID: 你的学号
- Date: 2024-10-24

---

## Part I: Image Matching

### Task 1: Find corners with harris detector

In [ ]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Let's load an example image
filename = 'building.jpeg'
img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
fig = plt.imshow(img, cmap='gray')

Step 1. Compute the covariance matrix at each point

$$H=\begin{bmatrix}I_x^2& I_xI_y\\I_xI_y &I_y^2\end{bmatrix}$$

where $I_x=\frac{\partial f}{\partial x}, I_y=\frac{\partial f}{\partial y}$.

In practice, we can further apply a 2D gaussian filter G to smooth H

$$ H'_{i,j,c} = \sum_{m=1}^{W} \sum_{n=1}^{W} G_{m,n} \cdot H_{i+m-\frac{W+1}{2},j+n-\frac{W+1}{2},c} $$

In [ ]:
# normalize image to [0,1]
img_float = ...

# use sobel operator at every location
sobel_x = ...
sobel_y = ...

# compute H 
H = np.zeros((img.shape[0], img.shape[1], 4))
H[..., 0] = ...
H[..., 1] = ...
H[..., 2] = ...
H[..., 3] = ...

# gaussian weights, what window-size should be used?
window_size = ...
gaussian_kernel_1d = cv2.getGaussianKernel(window_size, 0.5)
gaussian_kernel_2d = np.outer(gaussian_kernel_1d, gaussian_kernel_1d.transpose())
H = cv2.filter2D(H, -1, gaussian_kernel_2d)

Step 2. Harris response

Theoretically, we can compute eigenvalues

$H=\begin{bmatrix}a&b\\c&d\end{bmatrix}\quad \lambda_\pm=\frac{1}{2}((a+d)\pm\sqrt{4bc+(a-d)^2})$

and then classify points using eigenvalues of H, like:

<img src="https://opencv24-python-tutorials.readthedocs.io/en/latest/_images/harris_region.jpg" alt="drawing" width="200"/>

However, computing eigenvalues are expensive, so we use the following alternative:

$$f=\frac{\lambda_1 \lambda_2}{\lambda_1+\lambda_2}=\frac{determinant(H)}{trace(H)}$$

where $det(\begin{bmatrix}a&b\\c&d\end{bmatrix})=ad-bc$,   and $trace(\begin{bmatrix}a&b\\c&d\end{bmatrix})=a+d$

In [ ]:
# compute harris response
f = ...

Step 3. Threshold $f$ and visualize

we skip non-maximum suppression operation here. You only need to visualize the thresholded harris response map.

In [ ]:
# threshold and visualize the binary response map with plt.imshow(f_binary)
# the threshold is a hyperparameter, try different values
f_binary = ...
plt.imshow(f_binary)

Further reading:

[【计算机视觉】2. 特征点检测：Harris, SIFT, SURF, ORB](https://zhuanlan.zhihu.com/p/36382429)

### Task 2: SIFT

To do this task, read [opencv documentation on SIFT](https://docs.opencv.org/4.5.4/d7/d60/classcv_1_1SIFT.html) first, and use it for local feature detection and description.

In [ ]:
# load an image
img0 = cv2.imread("1.jpeg", cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread("2.jpeg", cv2.IMREAD_GRAYSCALE)

In [ ]:
# creat sift extractor (detector + descriptor)
SIFT = ...

# get the detector and descriptor
kpts0, descs0 = ...
kpts1, descs1 = ...

For SIFT descriptors, people usually match them with ratio-test.

(1) Please list the main advantage of ratio-test in matching SIFT descriptors.

(2) Do you think mutual-nearest-neighbor method can also work?

 <span style="color:red">
 
 
 REPLACE THIS WITH YOUR ANSWER
 
 
 </span>

In [ ]:
# compute descriptor distance
distance = ...

# ratio test
mkpts0, mkpts1 = ...

And visualize the final matches.

In [ ]:
# visualization
# You might need to read the documentation of this function. Or you can write your own drawing function.
from utils import make_matching_figure

fig = make_matching_figure(...)
fig

---

## Part II: Image Stitching

One application fo image matching is to stitch multiple images and get one panorama.

### Task 3: Transformation

Considering 2 images as input, you can use SIFT (provided by cv2) to find the transformation between them (implement it on your own).

In [ ]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# load images
img0_rgb = cv2.imread("1.jpeg", cv2.IMREAD_COLOR)[..., [2, 1, 0]]
img1_rgb = cv2.imread("2.jpeg", cv2.IMREAD_COLOR)[..., [2, 1, 0]]
img0_gray = cv2.cvtColor(img0, cv2.COLOR_RGB2GRAY)
img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)

In [ ]:
# compute SIFT keypoints and descriptors
# note: on gray image

# find matches
mkpts0 = ...
mkpts1 = ...

Here, the transformation $H$ is defined as 
$$\begin{bmatrix}x_0\\y_0\\1\end{bmatrix}=\begin{bmatrix}h_{11}&h_{12}&h_{13}\\h_{21}&h_{22}&h_{23}\\0&0&1\end{bmatrix}\begin{bmatrix}x_1\\y_1\\1\end{bmatrix}$$

Please answer:

(1) What type is this transformation?

(2) Please write down the converted equation in the form of $Ah=b$. To solve this equation, what's the minimal number of matches we need? 

 <span style="color:red">
 
 
 REPLACE THIS WITH YOUR ANSWER
 
 
 </span>

In [ ]:
# randomly select K matches (according to your answer)
selected_mkpts0 = ...
selected_mkpts1 = ...

# solve the equation
A = ...
h = ...

### Task 4: RANSAC

To use naive ransac algorithm, we need $N$ sample-points(样本点), to solve the model, we need $K$ sample-points as a minimal requirement. Then perform:

1. Randomly sample $K$ sample-points.
2. Fit the model with $K$ sample-points. Denoted as $\hat h$.
3. Compute error of other sample points according to $\hat h$. Count the inliers within some threshold.
4. Repeat $M$ times, the final $h$ is the $\hat h$ with most inliers. 

In [ ]:
# implement your own RANSAC
def ransac_to_estimate_H(samples, K, inlier_thr, M): ...


H = ransac_to_estimate_H()

In [ ]:
# use cv2.warpPerspective to put one image on the other
height, width, channels = img1_rgb.shape
dsize = (width * 2, height)
panorama = cv2.warpPerspective(img0_rgb, H, dsize)
panorama[:, :width] = img1_rgb

# use plt to visualize the results
plt.imshow(panorama[..., [2, 1, 0]])

Hot to solve the artifacts in the overlapping region? Name 2 possible methods.

 <span style="color:red">
 
 
 WRITE YOUR ANSWER HERE.
 
 
 </span>